In [1]:
import requests
import regex as re
import matplotlib.pyplot as plt

In [2]:

def read_books(url):
    r = requests.get(url)
    r.encoding = r.apparent_encoding

    # What comes back includes headers and other HTTP stuff, get just the body of the response
    t = r.text
    return t

#save all the books
the_extra_ordinary_adv_lupin = 'https://www.gutenberg.org/files/6133/6133-0.txt'
arsene_lupin = 'https://www.gutenberg.org/cache/epub/4014/pg4014.txt'
lupin_vs_herlock_sholmes = 'https://www.gutenberg.org/files/40203/40203-0.txt'
the_hollow_needle = 'https://www.gutenberg.org/files/4017/4017-0.txt'
the_confession_of_lupin = 'https://www.gutenberg.org/cache/epub/28093/pg28093.txt'


#read all the books one by one
book1 = read_books(the_extra_ordinary_adv_lupin)
book2 = read_books(arsene_lupin)
book3 = read_books(lupin_vs_herlock_sholmes)
book4 = read_books(the_hollow_needle)
book5 = read_books(the_confession_of_lupin)



In [15]:
# Split the text into chapters
def splitChapters(bookName):
  
  chapterTitles = []
  bookStart = 0

  for bookStart, line in enumerate(bookName.splitlines()):
    # if the table of contents has been read and chapter one appears
    # again, you are done
    if len(chapterTitles) != 0:
      if re.search(chapterTitles[0], line, flags=re.IGNORECASE):
        break
    # identify chapter number by roman numeral and add to list of chapter names
    if re.search(r'([IVX]\.)', line):
      title = re.search(r'\.\s*((\S+\s?)+)', line)
      if title:
        chapterTitles.append(title.group(1).rstrip())

  chapters = []
  isTitle = 0
  # iterate through each line of the text again
  for lineNum, line in enumerate(bookName.splitlines()):
    isTitle = 0
    if lineNum >= bookStart:
      # end loop at the end of the book
      if re.search(r'End of (the)? Project Gutenberg', line, flags=re.IGNORECASE):
        break  
      # each chapter starts with the title
      for i in range(len(chapterTitles)):
        pattern = '^([IVXL]*\.\s)?' + chapterTitles[i] + '\.*$'
        if re.search(pattern, line, flags=re.IGNORECASE):
          if re.search(r'[A-Z]', line[0]):
            chapters.append('')
            isTitle = 1
            break

      # append lines to each chapter except for chapter title
      if len(chapters) > 0 and isTitle == 0:
        chapters[len(chapters) - 1] += line
        chapters[len(chapters) - 1] += '\n'   

  return chapterTitles, chapters


# Split the text into sentences
def splitSentences(chapter):
  word = r"(([\p{L}\w]+([’']\w*)?"
  punc = r'[\s$%&\'’()*+,\-\/:;<=>@\[\\\]^_‘{|}~]'
  end  = r'*)+[\"\.!\?]+)'
  sentencePattern = word + punc + end
  sentences = re.findall(sentencePattern, chapter)
  return sentences

In [16]:
## Question 3: When is the perpetrator first mentioned - chapter #, the sentence(s) # in a chapter,
bookName = book1
chapterTitles, chapters = splitChapters(bookName)

# loop chapters                                        
for i, chapter in enumerate(chapters):
  sentences = splitSentences(chapter)
  # loop sentences
  for j, sentence in enumerate(sentences):
    # find Lupin's name in a sentence then print
    if re.search(r'(Arsène[\n\s]Lupin|Lupin|Arsène)', sentence[0]):
      print('Chapter: ', i+1, '\tSentence: ', j+1, '\n', sentence[0])
      break
  else:
    continue
  break

Chapter:  1 	Sentence:  17 
 Arsène Lupin is on your vessel, first cabin, blonde hair, wound right
fore-arm, traveling alone under name of R........


In [26]:
## Question 4: What are the 3 words that occur around the perpetrator on each mention (i.e., the three words preceding, and the three words following the mention of a perpetrator),
bookName = book1
chapterTitles, chapters = splitChapters(bookName)

before = []
after  = []
Lupin  = r"(Arsène[\s\n]Lupin|Lupin|Arsène)[’']?s?"

matches = []
a = []
b = []

# loop chapters
for i, chapter in enumerate(chapters):
  sentences = splitSentences(chapter)
  # loop sentences
  for j, sentence in enumerate(sentences):
    # find Lupin's nane
    if re.search(Lupin, sentence[0]):
      # split the sentence into parts not containing Lupin's name
      brokenSentence = re.split(Lupin, sentence[0])
      for k, partition in enumerate(brokenSentence):
        # when Lupin is found in a sentence...
        if re.search(Lupin, partition):
          # LOOK BEHIND him and find the 3 closest words within the same sentence
          index = k
          while index > 0:
            matches = re.findall(r"[\p{L}\w]+[’']?\w*?", brokenSentence[index-1])
            while len(matches) > 0:
              b.append(matches.pop())
              if len(b) >= 3: break
            if len(b) >= 3: break
            else: index -= 1
          
          # If there are no more words in the sentence, go to the previous sentence
          index = j
          while len(b) < 3 and index > 0:
            matches = re.findall(r"[\p{L}\w]+[’']?\w*?", sentences[index-1][0])
            while len(matches) > 0 and len(b) < 3:
              b.append(matches.pop())
            index -= 1
          b.reverse()
          before.append(b.copy())
          b.clear()

          # now LOOK IN FRONT and find 3 words
          index = k
          while index < len(brokenSentence) - 1:
            matches = re.findall(r"[\p{L}\w]+[’']?\w*?", brokenSentence[index+1])
            while len(matches) > 0:
              a.append(matches.pop(0))
              if len(a) >= 3: break
            if len(a) >= 3: break
            else: index += 1

          # if there are no more words in the sentence, go to the next sentence
          index = j
          while len(a) < 3 and index < len(sentences) - 1:
            matches = re.findall(r"[\p{L}\w]+[’']?\w*?", sentences[index+1][0])
            while len(matches) > 0 and len(a) < 3:
              a.append(matches.pop(0))
            index += 1
          after.append(a.copy())
          a.clear()
        
for i in range(len(after)):
  print(before[i], after[i], '\n')

['to', 'other', 'passengers'] ['is', 'on', 'your'] 

['name', 'under', 'which'] ['was', 'concealing', 'himself'] 

['that', 'the', 'famous'] ['was', 'hiding', 'in'] 

['in', 'our', 'midst'] ['in', 'our', 'midst'] 

['and', 'picturesque', 'surroundings'] ['gentleman', 'burglar', 'will'] 

['furniture', 'is', 'genuine'] ['the', 'man', 'of'] 

['this', 'startling', 'situation'] ['was', 'wandering', 'about'] 

['that', 'music', 'room'] ['was', 'perhaps', 'this'] 

['personal', 'enemy', 'of'] ['Oh', 'oh', 'you'] 

['the', 'first', 'place'] ['calls', 'himself', 'Monsieur'] 

['the', 'work', 'of'] ['That', 'day', 'at'] 

['birth', 'of', 'course'] ['will', 'furnish', 'you'] 

['a', 'man', 'like'] ['could', 'commit', 'a'] 

['the', 'discovery', 'of'] ['or', 'other', 'person'] 

['Rozaine', 'Rozaine', 'against'] ['or', 'rather', 'according'] 

['to', 'current', 'opinion'] ['himself', 'against', 'Arsène'] 

['Lupin', 'himself', 'against'] ['the', 'contest', 'promised'] 

['is', 'one', 'exception'

In [4]:
# BOOK 3 EXCLUSIVELY

